In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.2 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 8.5 MB/s eta 0:00:00


In [6]:
model="meta-llama/Llama-2-7b-chat-hf"

In [7]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-665ec903-30b9ba3818697e436c335227;04e0589c-be14-4fee-8c88-ce28af8fab39)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

In [8]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-665ec909-5b27438f2d2d3b720f2d0aa9;17ca6f4f-50df-4b68-bb25-2413f35ce5d0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

In [9]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

NameError: name 'tokenizer' is not defined

In [10]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

NameError: name 'pipeline' is not defined

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

In [ ]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [ ]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [ ]:
instruction = "Who won the T20 Cricket World Cup 2022"

In [ ]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [ ]:
SYSTEM_PROMPT=B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS

In [ ]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [ ]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [ ]:
text ="How are you"

In [ ]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [ ]:
LLM_Chain.run(text)

In [ ]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [ ]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

In [ ]:
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS

In [ ]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [ ]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [ ]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)
LLM_Chain.run(text)